In [104]:
%load_ext autoreload
%autoreload 2

import re
import nltk
import numpy as np
import pandas as pd
import os
from pronounResolution import pronResolution_base, pronEval
from relationExtract import simpleRE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
files = [x for x in os.listdir('prep_scripts') if '_gapi' in x]
df = pd.read_csv('prep_scripts/' + files[0])[['speaker', 'dialogue', 'sentences', 'sentiment', 'entities', 'tokens']]
df['tokens'] = df['tokens'].apply(lambda x: eval(x))
df['sentiment'] = df['sentiment'].apply(lambda x: eval(x))
df.head(10)

,speaker,dialogue,sentences,sentiment,entities,tokens
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'magnitude': 0.3, 'score': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'pos': 'NUM', 'index': 0, 'content': '1989',..."
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'magnitude': 0.1, 'score': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'pos': 'NOUN', 'index': 0, 'content': 'Stark..."
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'magnitude': 0.6, 'score': -0.6}",[],"[{'pos': 'PRON', 'index': 3, 'content': 'He', ..."
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'magnitude': 1, 'score': -0.1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'pos': 'X', 'index': 2, 'content': 'Hello', ..."
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{'magnitude': 0.4, 'score': 0.4}","[{'salience': 0.34197578, 'meta': {}, 'type': ...","[{'pos': 'PRON', 'index': 1, 'content': 'I', '..."
5,Peggy Carter,Tell me that isn't what I think it is.,"[{'content': u""Tell me that isn't what I think...","{'magnitude': 0.6, 'score': -0.6}",[],"[{'pos': 'VERB', 'index': 0, 'content': 'Tell'..."
6,Dr. Hank Pym,"It depends, if you think it's a poor attempt t...","[{'content': u""It depends, if you think it's a...","{'magnitude': 1, 'score': -0.5}","[{'salience': 0.88545346, 'meta': {}, 'type': ...","[{'pos': 'PRON', 'index': 1, 'content': 'It', ..."
7,Mitchell Carson,You were instructed to go to Russia. May I rem...,[{'content': u'You were instructed to go to Ru...,"{'magnitude': 0.7, 'score': 0}","[{'salience': 0.87446207, 'meta': {}, 'type': ...","[{'pos': 'PRON', 'index': 2, 'content': 'You',..."
8,Dr. Hank Pym,I'm a scientist.,"[{'content': u""I'm a scientist."", 'begin': 0, ...","{'magnitude': 0.3, 'score': 0.3}","[{'salience': 1, 'meta': {}, 'type': u'PERSON'...","[{'pos': 'PRON', 'index': 1, 'content': 'I', '..."
9,Howard Stark,Then act like one. The Pym Particle is the mos...,"[{'content': u'Then act like one.', 'begin': 0...","{'magnitude': 1.2, 'score': 0.1}","[{'salience': 0.6887065, 'meta': {}, 'type': u...","[{'pos': 'ADV', 'index': 1, 'content': 'Then',..."


In [31]:
cList = list(df.speaker.unique())
df['total_sent'] = df['sentiment'].apply(lambda x: x['score'] * x['magnitude'])
cDict = dict(df.groupby('speaker').total_sent.sum())
df.shift().head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'magnitude': 0.3, 'score': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'pos': 'NUM', 'index': 0, 'content': '1989',...",0.09
2,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'magnitude': 0.1, 'score': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'pos': 'NOUN', 'index': 0, 'content': 'Stark...",0.01
3,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'magnitude': 0.6, 'score': -0.6}",[],"[{'pos': 'PRON', 'index': 3, 'content': 'He', ...",-0.36
4,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'magnitude': 1, 'score': -0.1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'pos': 'X', 'index': 2, 'content': 'Hello', ...",-0.10


In [84]:
df.loc[436]

speaker                                            Dr. Hank Pym
dialogue                    She doesn't think that we need you.
sentences     [{'content': u"She doesn't think that we need ...
sentiment                          {'magnitude': 0, 'score': 0}
entities                                                     []
tokens        [{'char': ['Ice Cream Store Customer'], 'pos':...
total_sent                                                    0
hasChar                                                    True
Name: 436, dtype: object

In [105]:
pronEval([df, df], num=2)

*******for the following dialogue********
Dr. Hank Pym: You think I don't want to? I can't. I spent years wearing it. It took a toll on me. You're our only option. Before Hope lost her mother, she used to look at me like I was the greatest man in the world. And now she looks at me and it's just disappointment. It's too late for me, but not for you. This is your chance. The chance to earn that look in your daughter's eyes, to become the hero that she already thinks you are. It's not about saving our world, it's about saving theirs.
Scott Lang: Damn, that was a good speech.
=>Dr. Hank Pym: Scott, I need you to be the Ant-Man.<=
narrator: we see Scott wearing the Ant-Man suit with Pym and Hope training him
Dr. Hank Pym: In the right hands, the relationship between man and suit is symbiotic. The suit has power, the man harnesses that power. You need to be skillful, agile, and above all, you need to be fast. You should be able to shrink and grow on a dime, so your size always suits your nee

In [73]:
df['tokens'], df['hasChar'] = zip(*df.apply(lambda x:pronResolution_base(cList, x), axis=1).values)
df.head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent,hasChar
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'magnitude': 0.3, 'score': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'pos': 'NUM', 'index': 0, 'content': '1989',...",0.09,False
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'magnitude': 0.1, 'score': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'pos': 'NOUN', 'index': 0, 'content': 'Stark...",0.01,False
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'magnitude': 0.6, 'score': -0.6}",[],"[{'char': ['Police Radio'], 'pos': 'PRON', 'in...",-0.36,True
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'magnitude': 1, 'score': -0.1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'pos': 'X', 'index': 2, 'content': 'Hello', ...",-0.10,True
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{'magnitude': 0.4, 'score': 0.4}","[{'salience': 0.34197578, 'meta': {}, 'type': ...","[{'char': ['Pym Tech Security Guard'], 'pos': ...",0.16,True


In [35]:
df['relations'] = df.tokens.apply(lambda x:simpleRE(x))

In [37]:
df[df.relations.notnull()].head().relations.values[1]

[{'noun': 'narrator', 'obj': 'detour.', 'verb': 'took'},
 {'noun': 'narrator', 'obj': 'vial', 'verb': 'places'}]

In [27]:
df.head().tokens.apply(lambda x:simpleRE(x))

{'begin': 12, 'lemma': 'Pym', 'label': 'NSUBJ', 'index': 4, 'pos': 'NOUN', 'content': 'Pym'}
{'begin': 0, 'lemma': 'I', 'label': 'NSUBJ', 'index': 1, 'pos': 'PRON', 'char': 'narrator', 'content': 'I'}
{'begin': 17, 'lemma': 'he', 'label': 'NSUBJ', 'index': 6, 'pos': 'PRON', 'char': 'narrator', 'content': 'he'}


0    [{'obj': 'facility', 'verb': 'enters', 'noun':...
1                                                 None
2                                                 None
3                                                 None
4    [{'obj': 'detour.', 'verb': 'took', 'noun': 'n...
Name: tokens, dtype: object